In [ ]:
# Header cell
from __future__ import division
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

# Specific things needed
import time
import math
import sys

# Add parent directory to path
sys.path.append('../code/')
sys.path.append('../sim/')

# Import deft modules
import deft_1d
import simulate_data_1d
import utils

In [ ]:
# Simulate data
N = 100
data_type = 'wide'

# simulate data and get default deft settings
data, defaults = simulate_data_1d.run(data_type, N)

In [ ]:
# Deft parameter settings
G = 100
alpha = 3
bbox = [-10, 10]
periodic = False
Laplace = False
num_samples = 100
num_steps_per_sample = G
num_thermalization_steps = 10*G
fix_t_at_t_star = False
print_t = False
tollerance = 1E-3
resolution = 1E-1

# Do density estimation
results = deft_1d.run(data, G=G, alpha=alpha, bbox=bbox, periodic=periodic, Laplace=Laplace, num_samples=num_samples, \
                      num_steps_per_sample=num_steps_per_sample, num_thermalization_steps=num_thermalization_steps, \
                      fix_t_at_t_star=fix_t_at_t_star, print_t=print_t, tollerance=tollerance, resolution=resolution)

In [ ]:
results.t_star

In [ ]:
# Plot prob(t) vs t
if (num_samples > 0):
    plt.figure(figsize=[8,6])
    x = results.prob_t_vs_t[0,:]
    y = results.prob_t_vs_t[1,:]
    plt.scatter(x, y)
    plt.semilogy(x, y, color='red', linewidth=1)
    plt.ylim(0, 1.05*max(y))
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.xlabel('t (-inf, +inf)', size=20)
    plt.ylabel('prob(t)', size=20)

In [ ]:
# Compute true density
xs = results.bin_centers
Q_true = np.zeros(G)
for i, x in enumerate(xs):
    Q_true[i] = eval(defaults['pdf_py'])
Q_true /= results.h*sum(Q_true)

In [ ]:
plt.figure(figsize=[16,10])
xs = results.bin_centers

# plot histogram density
left_bin_edges = results.bin_edges[:-1]
plt.bar(left_bin_edges, results.R, width=results.h, color='gray', linewidth=0, zorder=0, alpha=0.5)

# Plot the MCMC samples from S
if (num_samples > 0):
    plt.plot(xs, results.Q_samples, color='blue', linewidth=0.5, alpha=0.5)

# Plot DEFT density estimate
plt.plot(xs, results.Q_star, color='red', linewidth=2, zorder=2, alpha=1)

# Plot the true density
plt.plot(xs, Q_true, color='black', linewidth=2)

# Tidy up the plot
plt.ylim(0, 1.05*max(results.R))
plt.xlim(-10, 10)
plt.yticks(size=20)
plt.xticks(size=20)
plt.ylabel('Probability density', size=25)
plt.xlabel('Data', size=25)
t = results.deft_1d_compute_time
plt.title('%s, t=%1.2f sec %s'%(data_type, t, '(Black=Q_true, Red=Q_star, Blue=Q_samples)'), \
          fontsize=20)